In [ ]:
# add path / import
import sys
repo_root = "../"
if repo_root not in sys.path:
    sys.path.insert(0, repo_root)

from rdkit import RDLogger
from utils import conf_from_yaml, generator_from_conf
RDLogger.DisableLog('rdApp.*')

In [ ]:
# start generation
yaml_path = "config/generation/mcts_smiles.yaml"

conf = conf_from_yaml(yaml_path, repo_root)
generator = generator_from_conf(conf, repo_root)
generator.generate(time_limit=conf.get("time_limit"), max_generations=conf.get("max_generations"))

generator.plot(**conf.get("plot_args", {}))
generator.analyze()

In [ ]:
# continue generation
generator.generate(time_limit=60, max_generations=200)
generator.plot(**conf.get("plot_args", {}))
generator.analyze()

In [ ]:
# save mcts
save_path = generator.output_dir() + "save.mcts"
generator.save(save_path)

In [ ]:
# load generator and yaml
from generator import Generator
load_dir = "generation_result/08-04_13-54/"
conf = conf_from_yaml("sandbox/" + load_dir + "config.yaml", repo_root)
generator = generator_from_conf(conf, repo_root)
generator = Generator.load(load_dir + "save.mcts", generator.transition)

In [ ]:
# time test
import cProfile
import pstats

profiler = cProfile.Profile()
profiler.enable()

conf = conf_from_yaml(yaml_path, repo_root)
generator = generator_from_conf(conf, repo_root)
generator.generate(max_generations=1000)

profiler.disable()

stats = pstats.Stats(profiler)
stats.sort_stats("cumtime").print_stats(100)